In [ ]:
import json

import pandas as pd

class STATUS_LOADER:
    """
    STATUS_LOADER : 상태 정보 추출 클래스
    """

    def __init__(self, dict_df_result: dict, dict_df_position: dict) -> None:
        """
        STATUS_LOADER의 생성자

        :param dict dict_df_result: dict_df_result 입니다.
        :param dict dict_df_position: dict_df_position 입니다.
        :return: None
        :rtype: None
        """
        self.dict_df_result = dict_df_result
        self.dict_df_position = dict_df_position

    def get_current_cash(self) -> float:
        """
        현재 보유 cash를 반환하는 메서드

        :return : 현재 보유 cash
        :rtype: float
        """
        _dict_df_result = self.dict_df_result
        try:
            _df_result_total = _dict_df_result["TOTAL"]
            _current_cash = (
                _df_result_total.sort_values("DATE").tail(1)["CASH"].values[0]
            )
            return _current_cash
        except:
            return 1_000_000_000.0

    def get_status_df(self) -> pd.DataFrame:
        """
        현재 보유 position 관련 정보를 반환하는 메서드

        :return: 보유한 symbol, 갯수, 구매가격, 현재 가격을 데이터프레임 형태로 가져옵니다.
        :rtype: pd.DataFrame
        """
        current_symbol_list = list()
        _dict_df_result = self.dict_df_result
        _dict_df_position = self.dict_df_position

        _total_symbols = sorted(_dict_df_position.keys())

        for _symbol in _total_symbols:
            try:
                _symbol_result_df = _dict_df_result[_symbol]
                _symbol_position_df = _dict_df_position[_symbol]

                _current_price = (
                    _symbol_result_df.sort_values("DATE").tail(1)["PRICE"].values[0]
                )
                _trade_price = _symbol_position_df["TRADE_PRICE"].values[0]
                _current_qty = _symbol_position_df["QTY"].values[0]

                current_symbol_list.append(
                    {
                        "SYMBOL": _symbol,
                        "CURRENT_QTY": _current_qty,
                        "CURRENT_PRICE": _current_price,
                        "TRADE_PRICE": _trade_price,
                    }
                )
            except:
                pass
        return pd.DataFrame(
            current_symbol_list,
            columns=["SYMBOL", "CURRENT_QTY", "CURRENT_PRICE", "TRADE_PRICE"],
        )

In [ ]:
import datetime as dt

import pandas as pd

import kquant as kq

class SYMBOL_LOADER:
    """
    SYMBOL_LOADER : 거래가능한 주식 symbol을 필터-추출하는 클래스
    """

    @staticmethod
    def load_symbols_df() -> pd.DataFrame:
        """
        한국거래소 종목 목록 데이터프레임을 호출하는 메서드

        :return : 한국거래소 종목 목록 데이터프레임
        :rtype : pd.DataFrame
        """
        symbols_df = kq.symbol_stock()
        return symbols_df

    class SYMBOL_FILTER:
        """
        SYMBOl_FILTER : symbols를 filtering 하는 클래스
        """

        @staticmethod
        def filter__market(symbols_df: pd.DataFrame) -> pd.DataFrame:
            """
            market에 대한 필터링을 진행하는 메서드

            :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
            :return: MARKET이 [코스닥, 유가증권]에 속하는 데이터프레임
            :rtype: pd.DataFrame
            """
            filtered_symbols_df = symbols_df[
                (symbols_df["MARKET"].isin(["코스닥", "유가증권"]))
            ].copy()
            return filtered_symbols_df

        @staticmethod
        def filter__admin_issue(symbols_df: pd.DataFrame) -> pd.DataFrame:
            """
            ADMIN_ISSUE에 대한 필터링을 진행하는 메서드

            :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
            :return: ADMIN_ISSUE가 0인 데이터프레임
            :rtype: pd.DataFrame
            """
            filtered_symbols_df = symbols_df[(symbols_df["ADMIN_ISSUE"] == 0)].copy()
            return filtered_symbols_df

        @staticmethod
        def filter_sec_type(symbols_df: pd.DataFrame) -> pd.DataFrame:
            """
            SEC_TYPE에 대한 필터링을 진행하는 메서드

            :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
            :return: SEC_TYPE이 [ST, EF, EN]에 속하는 데이터프레임
            :rtype: pd.DataFrame
            """
            filtered_symbols_df = symbols_df[
                symbols_df["SEC_TYPE"].isin(["ST", "EF", "EN"])
            ].copy()
            return filtered_symbols_df

    def filter_symbols_df(self, symbols_df: pd.DataFrame) -> pd.DataFrame:
        """
        symbol_df 에 대한 필터링을 진행하는 메서드

        :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
        :return: SYMBOL_FILTER의 메서드를 거친 데이터프레임
        :rtype: pd.DataFrame
        """
        symbol_filter = self.SYMBOL_FILTER()
        filtered_symbols_df = symbol_filter.filter__market(symbols_df)
        filtered_symbols_df = symbol_filter.filter__admin_issue(filtered_symbols_df)
        filtered_symbols_df = symbol_filter.filter_sec_type(filtered_symbols_df)
        return filtered_symbols_df

    @staticmethod
    def get_symbols(symbols_df: pd.DataFrame) -> list:
        """
        symbols_df의 symbol을 중복을 제거하여 추출하는 메서드

        :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
        :return: symbols
        :rtype: list
        """
        symbols = sorted(set(symbols_df["SYMBOL"]))
        return symbols

    # SYMBOL_LOADER PIPELINE
    def __call__(self) -> list:
        """
        SYMBOL_LOADER의 파이프라인을 제공하는 메서드

        :return: 필터를 거친 symbols
        :rtype: list
        """
        symbols_df = self.load_symbols_df()
        filtered_symbols_df = self.filter_symbols_df(symbols_df)
        symbols = self.get_symbols(filtered_symbols_df)
        return symbols


class FUNDAMENTAL_LOADER:
    """
    FUNDAMENTAL_LOADER : fundamental_analysis를 위한 정보를 추출하는 클래스
    """

    def __init__(self, symbol: str, date: dt.date) -> None:
        """
        FUNDAMENTAL_LOADER의 생성자

        :param str symbol: stock의 symbol 입니다.
        :param datetime.date date: 현재 날짜 입니다.

        :attr : pd.DataFrmae daily_stock_df : 현재 날짜 기준 가장 최근 stock데이터 입니다.
        """
        self.symbol = symbol
        self.date = date
        self.daily_stock_df = kq.daily_stock(
            symbol,
            start_date=date - dt.timedelta(days=7),
            end_date=date,
        )

    def load_recent_close(self) -> float:
        """
        가장 최근 종가를 추출합니다.
        :return: 종가
        :rtype: float
        """
        daily_stock_df = self.daily_stock_df
        _close = daily_stock_df.sort_values("DATE").tail(1)["CLOSE"].values[0]
        return _close

    def load_recent_marketcap(self) -> float:
        """
        가장 최근 시가총액을 추출합니다.

        :return: 시가총액
        :rtype: float
        """
        daily_stock_df = self.daily_stock_df
        _marketcap = daily_stock_df.sort_values("DATE").tail(1)["MARKETCAP"].values[0]
        return float(_marketcap)

    def load_recent_netprofit(self) -> float:
        """
        공시자료 중 가장 최근 당기순이익을 추출합니다.

        :return: 당기순이익
        :rtype: float
        """
        netprofit_df = kq.account_history(
            symbol=self.symbol, account_code="122700", period="q"
        )
        netprofit_df.sort_values("YEARMONTH", inplace=True)
        _netprofit = netprofit_df.tail(1)["VALUE"].values[0] * 1000
        return float(_netprofit)

    def load_recent_assets(self) -> float:
        """
        공시자료 중 가장 최근 총 자산를 추출합니다.

        :return: 총 자산
        :rtype: float
        """
        assets_df = kq.account_history(
            symbol=self.symbol, account_code="111000", period="q"
        )
        assets_df.sort_values("YEARMONTH", inplace=True)
        _assets = assets_df.tail(1)["VALUE"].values[0] * 1000
        return float(_assets)

    def load_recent_current_assets(self) -> float:
        """
        공시자료 중 가장 최근 유동 자산를 추출합니다.

        :return: 유동 자산
        :rtype: float
        """
        current_assets_df = kq.account_history(
            symbol=self.symbol, account_code="111100", period="q"
        )
        current_assets_df.sort_values("YEARMONTH", inplace=True)
        _current_assets = current_assets_df.tail(1)["VALUE"].values[0] * 1000
        return float(_current_assets)

    def load_recent_liabilities(self) -> float:
        """
        공시자료 중 가장 최근 총 부채를 추출합니다.

        :return: 총 부채
        :rtype: float
        """
        liabilities_df = kq.account_history(
            symbol=self.symbol, account_code="113000", period="q"
        )
        liabilities_df.sort_values("YEARMONTH", inplace=True)
        _liabilities = liabilities_df.tail(1)["VALUE"].values[0] * 1000
        return float(_liabilities)

    def load_recent_equity(self) -> float:
        """
        공시자료 중 가장 최근 총 자본(총 자산 - 총 부채)를 추출합니다.

        :return: 총 자본(총 자산 - 총 부채)
        :rtype: float
        """
        equity_df = kq.account_history(
            symbol=self.symbol, account_code="115000", period="q"
        )
        equity_df.sort_values("YEARMONTH", inplace=True)
        _equity = equity_df.tail(1)["VALUE"].values[0] * 1000
        return float(_equity)

    def load_recent_EBITDA(self) -> float:
        """
        공시자료 중 가장 최근 EBITDA를 추출합니다.

        :return: EBITDA
        :rtype: float
        """
        ebitda_df = kq.account_history(
            symbol=self.symbol, account_code="123000", period="q"
        )
        ebitda_df.sort_values("YEARMONTH", inplace=True)
        _ebitda = ebitda_df.tail(1)["VALUE"].values[0] * 1000
        return float(_ebitda)

    def __call__(self) -> dict:
        """
        fundmanetal analysis를 위해 필요한 데이터를 가져와서 dictionary를 반환한다.

        :return: fundamental analysis를 위한 데이터 dictionary
        :rtype: dict
        """
        _close = self.load_recent_close()
        _marketcap = self.load_recent_marketcap()
        _netprofit = self.load_recent_netprofit()
        _assets = self.load_recent_assets()
        _equity = self.load_recent_equity()
        return {
            "SYMBOL": self.symbol,
            "CLOSE": _close,
            "MARKETCAP": _marketcap,
            "NETPROFIT": _netprofit,
            "ASSETS": _assets,
            "EQUITY": _equity,
        }

In [ ]:
import json

import pandas as pd

class SYMBOL_SECTOR_PROCESSOR:
    """
    SYMBOL_SECTOR_PROCESSOR : symbol의 sector_code를 찾아내는 클래스
    """

    def __init__(self, symbols) -> None:
        """
        SYMBOL_SECTOR_PROCESSOR의 생성자

        :param list symbols: sector_code를 찾을 symbol들
        """
        self.symbols = symbols

    @staticmethod
    def load_symbol_sector_dict() -> dict:
        """
        symbol:sector_code 딕셔너리를 읽어오는 메서드

        :return: symbol-sector_code 딕셔너리
        :rtype: dict
        """
        with open("./data/symbol_sector_dict.json", "r") as f:
            symbol_sector_dict = json.load(f)
        return symbol_sector_dict

    @staticmethod
    def format_symbol_df(symbols) -> pd.DataFrame:
        """
        symbol_df를 생성하는 메서드

        :param list symbols: sector_code를 찾을 symbol들
        :return: symbol의 데이터프레임
        :rtype: pd.DataFrame
        """
        symbol_df = pd.DataFrame(symbols, columns=["SYMBOL"])
        return symbol_df

    @staticmethod
    def append_sector(symbol_df, symbol_sector_dict) -> pd.DataFrame:
        """
        symbol_df에 sector를 추가하는 메서드

        :param pd.DataFrame symbol_df: symbol의 데이터프레임
        :param dict symbol_sector_dict: symbol:sector의 딕셔너리
        :return: sector가 추가된 symbol_df
        :rtype: pd.DataFrame
        """
        symbol_df["SECTOR"] = symbol_df["SYMBOL"].map(symbol_sector_dict)
        return symbol_df

    @staticmethod
    def get_filtered_sectors(symbol_df, n) -> list:
        """
        symbol_df를 필터링 하여 sector를 제공하는 메서드

        :param pd.DataFrame symbol_df: symbol의 데이터프레임
        :param int n: n개 이하의 symbol이 있는 sector 제거
        :return: 필터링 된 symbol_df의 sector
        :rtype: list
        """
        symbol_groupby = symbol_df.groupby("SECTOR")
        filtered_sectors = symbol_groupby.count()["SYMBOL"][
            symbol_groupby.count()["SYMBOL"] > n
        ].index
        return filtered_sectors

    @staticmethod
    def get_filtered_symbol_df(symbol_df, filtered_sectors) -> pd.DataFrame:
        """
        symbol_df를 필터링 하는 메서드

        :param pd.DataFrame symbol_df: symbol의 데이터프레임
        :param list filtered_sectors: filtered된 sector들
        :return: 필터링 된 symbol_df
        :rtype: pd.DataFrame
        """
        filtered_symbol_df = symbol_df[symbol_df["SECTOR"].isin(filtered_sectors)]
        return filtered_symbol_df

    @staticmethod
    def get_sampled_symbol_df(filtered_symbol_df, n) -> pd.DataFrame:
        """
        각 sector별로 n개를 sampling하는 메서드

        :param pd.DataFrame filtered_symbol_df: filter된 symbol_df
        :param int n: 샘플링 갯수
        :return: 샘플링 된 symbol_df
        :rtype: pd.DataFrame
        """
        sampled_symbol_df = filtered_symbol_df.groupby("SECTOR").sample(n)
        return sampled_symbol_df

    def __call__(self) -> pd.DataFrame:
        """
        SYMBOL_SECTOR_PROCESSOR의 파이프라인을 제공하는 메서드

        :return: 샘플링 된 symbol_df
        :rtype: pd.DataFrame
        """
        symbols = self.symbols
        symbol_sector_dict = self.load_symbol_sector_dict()

        symbol_df = self.format_symbol_df(symbols=symbols)
        symbol_df = self.append_sector(
            symbol_df=symbol_df, symbol_sector_dict=symbol_sector_dict
        )

        filtered_sectors = self.get_filtered_sectors(symbol_df=symbol_df, n=30)
        filtered_symbol_df = self.get_filtered_symbol_df(
            symbol_df=symbol_df, filtered_sectors=filtered_sectors
        )
        sampled_symbol_df = self.get_sampled_symbol_df(
            filtered_symbol_df=filtered_symbol_df, n=15
        )
        return sampled_symbol_df


In [ ]:
import json

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from ..loader.api_loader import FUNDAMENTAL_LOADER


class PBR_PROCESSOR:
    """
    PBR_PROCESSOR : PBR에 대한 정보를 SCORE로 정제하여 반환하는 클래스
    """

    def __init__(self, fundamental_df) -> None:
        self.fundamental_df = fundamental_df

    @staticmethod
    def append_pbr(fundamental_df) -> pd.DataFrame:
        """
        PBR을 추가하는 메서드

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        :return: PBR가 추가된 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df["PBR"] = fundamental_df["MARKETCAP"] / (fundamental_df["EQUITY"])
        return fundamental_df

    @staticmethod
    def filter_negative_pbr(fundamental_df) -> pd.DataFrame:
        """
        PBR이 음수인 데이터를 제거하는 메서드

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        :return: PBR이 음수가 아닌 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df = fundamental_df[fundamental_df["PBR"] > 0]
        return fundamental_df

    @staticmethod
    def append_score(fundamental_df: pd.DataFrame) -> pd.DataFrame:
        """
        PBR을 기준으로, 낮은 PER일 수록 큰 SCORE를 주는 메서드

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        :return: SCORE가 추가된 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df["PBR_SCORE"] = (
            fundamental_df["PBR"].sum() / fundamental_df["PBR"]
        )
        return fundamental_df

    def __call__(self) -> pd.DataFrame:
        """
        PBR기반의 score 추가한 데이터프레임을 반환하는 메서드

        :return: SCORE가 추가된 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df = self.fundamental_df

        fundamental_df = self.append_pbr(fundamental_df)
        fundamental_df = self.filter_negative_pbr(fundamental_df)
        fundamental_df = self.append_score(fundamental_df)

        score_df = fundamental_df.loc[:, ["SYMBOL", "PBR_SCORE"]]
        return score_df


class PER_PROCESSOR:
    """
    PER_PROCESSOR : PER에 대한 정보를 SCORE로 정제하여 반환하는 클래스
    """

    def __init__(self, fundamental_df) -> None:
        """
        PER_PROCESSOR의 생성자

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        """
        self.fundamental_df = fundamental_df

    @staticmethod
    def append_per(fundamental_df) -> pd.DataFrame:
        """
        PER을 추가하는 메서드

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        :return: PER가 추가된 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df["PER"] = fundamental_df["MARKETCAP"] / (
            fundamental_df["NETPROFIT"]
        )
        return fundamental_df

    @staticmethod
    def filter_negative_per(fundamental_df) -> pd.DataFrame:
        """
        PER이 음수인 데이터를 제거하는 메서드

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        :return: PER이 음수가 아닌 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df = fundamental_df[fundamental_df["PER"] > 0]
        return fundamental_df

    @staticmethod
    def append_score(fundamental_df: pd.DataFrame) -> pd.DataFrame:
        """
        PER을 기준으로, 낮은 PER일 수록 큰 SCORE를 주는 메서드

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        :return: SCORE가 추가된 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df["PER_SCORE"] = (
            fundamental_df["PER"].sum() / fundamental_df["PER"]
        )
        return fundamental_df

    def __call__(self) -> pd.DataFrame:
        """
        PER기반의 score 추가한 데이터프레임을 반환하는 메서드

        :return: SCORE가 추가된 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df = self.fundamental_df

        fundamental_df = self.append_per(fundamental_df)
        fundamental_df = self.filter_negative_per(fundamental_df)
        fundamental_df = self.append_score(fundamental_df)

        score_df = fundamental_df.loc[:, ["SYMBOL", "PER_SCORE"]]
        return score_df


from typing import Any


class SCORE_PROCESSOR:
    """
    SCORE_PROCESSOR : PBR / PER을 종합하여 SCORE 데이터를 제공하는 클래스
    """

    def __init__(self, symbols, date, CFG={"pbr_ratio": 1, "per_ratio": 0.3}) -> None:
        """
        SCORE_PROCESSOR의 생성자

        :param list symbols: score 확인할 symbols
        :param datetime.date date: 매매일 날짜
        :param dict  CFG: score_processor 파라미터
        """
        self.symbols = symbols
        self.date = date
        self.CFG = CFG

    @staticmethod
    def load_fundamental_df(symbols, date) -> pd.DataFrame:
        """
        기본적 분석을 위한 fundamental_df를 load하는 메서드

        :param list symbols: score 확인할 symbols
        :param datetime.date date: 매매일 날짜
        :return: 기본적 분석을 위한 데이터
        :rtype: pd.DataFrame
        """
        fundamental_data_list = list()
        for symbol in symbols:
            try:
                _fundamental_loader = FUNDAMENTAL_LOADER(symbol, date)
                _fundamental_data = _fundamental_loader()
                fundamental_data_list.append(_fundamental_data)
            except:
                pass
        fundamental_df = pd.DataFrame(fundamental_data_list)
        return fundamental_df

    @staticmethod
    def get_symbol_close_dict(fundamental_df) -> dict:
        """
        종목:종가 사전을 추출하는 메서드

        :param pd.DataFrame fundamental_df: 기본적 분석 관련 데이터
        :return: {종목:종가}의 dictionary
        :rtype: dict
        """
        symbol_close_dict = fundamental_df.set_index("SYMBOL")["CLOSE"].to_dict()
        return symbol_close_dict

    @staticmethod
    def get_pbr_score_df(fundamental_df) -> pd.DataFrame:
        """
        pbr 스코어를 제공하는 메서드

        :param pd.DataFrame fundamental_df: 기본적 분석 관련 데이터
        :return: pbr 점수가 있는 데이터
        :rtype: pd.DataFrame
        """
        pbr_processor = PBR_PROCESSOR(fundamental_df)
        pbr_score_df = pbr_processor()
        return pbr_score_df

    @staticmethod
    def get_per_score_df(fundamental_df) -> pd.DataFrame:
        """
        per 스코어를 제공하는 메서드

        :param pd.DataFrame fundamental_df: 기본적 분석 관련 데이터
        :return: per 점수가 있는 데이터
        :rtype: pd.DataFrame
        """
        per_processor = PER_PROCESSOR(fundamental_df)
        per_score_df = per_processor()
        return per_score_df

    @staticmethod
    def format_score_df(
        symbol_close_dict, pbr_score_df, per_score_df, CFG
    ) -> pd.DataFrame:
        """
        score_df를 반환하는 메서드

        :param dict symbol_close_dict: 종목의 종가를 가진 딕셔너리
        :param pd.DataFrame pbr_score_df: pbr 점수 관련 데이터
        :param pd.DataFrame per_score_df: per 점수 관련 데이터
        :param dict CFG: pbr, per의 weight 파라미터를 위한 딕셔너리

        :return: 총합(pbr,per) 데이터
        :rtype: pd.DataFrame
        """
        score_df = pbr_score_df.merge(
            per_score_df.loc[:, ["SYMBOL", "PER_SCORE"]], on="SYMBOL"
        )

        mms = MinMaxScaler()
        score_df.iloc[:, 1:] = mms.fit_transform(score_df.iloc[:, 1:])

        score_df["SCORE"] = (
            score_df["PBR_SCORE"] * CFG["pbr_ratio"]
            + score_df["PER_SCORE"] * CFG["per_ratio"]
        )
        score_df["CLOSE"] = score_df["SYMBOL"].map(symbol_close_dict)
        return score_df

    def __call__(self) -> pd.DataFrame:
        """
        SCORE_PROCESSOR의 파이프라인을 제공하는 메서드

        :return: 총합(pbr,per) 데이터
        :rtype: pd.DataFrame
        """
        symbols = self.symbols
        date = self.date
        CFG = self.CFG

        fundamental_df = self.load_fundamental_df(symbols, date)
        symbol_close_dict = self.get_symbol_close_dict(fundamental_df)

        pbr_score_df = self.get_pbr_score_df(fundamental_df)
        per_score_df = self.get_per_score_df(fundamental_df)

        score_df = self.format_score_df(
            symbol_close_dict, pbr_score_df, per_score_df, CFG
        )
        return score_df

In [ ]:
import numpy as np
import pandas as pd


class BUYING_ORDER_PROCESSOR:
    """
    BUYING_ORDER_PROCESSOR : 매수주문을 생성하는 클래스
    """

    def __init__(self, score_df, invest_money, status_df, n) -> None:
        """
        BUYING_ORDER_PROCESSOR의 생성자

        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param float invest_money: 당일 활용 투자 금액
        :param pd.DataFrame status_df: 현재 position과 관련된 정보를 가진 데이터프레임
        :param int n: 당일 투자종목의 갯수
        """
        self.score_df = score_df
        self.invest_money = invest_money
        self.status_df = status_df
        self.n = n

    @staticmethod
    def filter_positioned_symbol(score_df, positioned_symbol) -> pd.DataFrame:
        """
        이미 position이 있는 종목을 필터링 하는 메소드

        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param list positioned_symbol: 현재 이미 position이 있는 symbol의 리스트
        """
        filtered_score_df = score_df[~(score_df["SYMBOL"].isin(positioned_symbol))]
        return filtered_score_df

    @staticmethod
    def get_filtered_score_df(score_df, n) -> pd.DataFrame:
        """
        score_df의 상위 n개의 row를 추출하는 메서드

        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param int n: 당일 투자종목의 갯수
        """
        high_limit = np.percentile(score_df["SCORE"], 95)
        low_limit = np.percentile(score_df["SCORE"], 80)

        filtered_score_df = score_df[
            (score_df["SCORE"] < high_limit) & (score_df["SCORE"] > low_limit)
        ].nlargest(n, "SCORE")

        return filtered_score_df

    @staticmethod
    def append_price_invest(high_score_df, invest_money) -> pd.DataFrame:
        """
        당일 활용 투자 금액을 score 기준으로 분배한 column을 생성하는 메서드

        :param pd.DataFrame high_score_df: score_df의 score 상위 데이터프레임
        :param float invest_money: 당일 활용 투자 금액
        """
        high_score_df["PRICE_INVEST"] = (
            high_score_df["SCORE"] / high_score_df["SCORE"].sum()
        ) * invest_money
        return high_score_df

    @staticmethod
    def append_cnt_invest(high_score_df: pd.DataFrame) -> pd.DataFrame:
        """
        당일 활용 투자 금액 최근 종가로 나누어 주문 갯수를 column으로 생성하는 메서드

        :param pd.DataFrame high_score_df: score_df의 score 상위 데이터프레임
        """
        high_score_df["CNT_INVEST"] = (
            high_score_df["PRICE_INVEST"] // high_score_df["CLOSE"]
        )
        return high_score_df

    @staticmethod
    def get_order_from_df(df) -> list:
        """
        데이터 프레임에서 signiture에 맞게 주문 list를 추출하는 메서드

        :param pd.DataFrame df: [SYMBOL,CNT_INVEST]를 가진 데이터프레임
        """
        orders = list(
            df.set_index("SYMBOL")["CNT_INVEST"].astype(int).to_dict().items()
        )
        return orders

    def __call__(self) -> list:
        """
        BUYING_ORDER_PROCESSOR의 pipeline을 진행하는 메서드

        """
        score_df = self.score_df
        invest_money = self.invest_money
        status_df = self.status_df
        n = self.n

        positioned_symbol = sorted(set(status_df["SYMBOL"]))
        filtered_positioned_df = self.filter_positioned_symbol(
            score_df, positioned_symbol
        )

        filtered_score_df = self.get_filtered_score_df(filtered_positioned_df, n)
        filtered_score_df = self.append_price_invest(filtered_score_df, invest_money)
        filtered_score_df = self.append_cnt_invest(filtered_score_df)
        buying_order = self.get_order_from_df(filtered_score_df)
        return buying_order


class SELLING_ORDER_PROCESSOR:
    """
    SELLING_ORDER_PROCESSOR : 매도주문을 추출하는 클래스
    """

    def __init__(
        self,
        status_df,
        CFG={
            "upper_limit": 9,
            "lower_limit": -3,
        },
    ) -> None:
        """
        SELLING_ORDER_PROCESSOR의 생성자

        :param pd.DataFrame status_df: 현재 position과 관련된 정보를 가진 데이터프레임
        :param dict CFG: 매도로직을 위한 한계선 dictionary
        """
        self.status_df = status_df
        self.CFG = CFG

    @staticmethod
    def append_profit_loss(status_df) -> pd.DataFrame:
        def calc_profit_loss(trade_price, current_price):
            profit_loss = ((current_price - trade_price) / trade_price) * 100
            return profit_loss

        status_df["PROFIT_LOSS"] = status_df.apply(
            lambda x: calc_profit_loss(x.TRADE_PRICE, x.CURRENT_PRICE), axis=1
        )
        return status_df

    @staticmethod
    def get_filter_status_df(status_df, CFG) -> pd.DataFrame:
        filter_status_df = status_df[
            (status_df["PROFIT_LOSS"] > CFG["upper_limit"])
            | (status_df["PROFIT_LOSS"] < CFG["lower_limit"])
        ]
        return filter_status_df

    @staticmethod
    def get_order_from_df(df) -> list:
        """
        데이터 프레임에서 signiture에 맞게 주문 list를 추출하는 메서드

        :param pd.DataFrame df: [SYMBOL,CURRENT_QTY]를 가진 데이터프레임
        """
        orders = list(
            df.set_index("SYMBOL")["CURRENT_QTY"]
            .apply(lambda x: x * -1)
            .astype(int)
            .to_dict()
            .items()
        )
        return orders

    def __call__(self) -> list:
        """
        SELLING_ORDER_PROCESSOR의 pipeline을 진행하는 메서드
        """
        status_df = self.status_df
        CFG = self.CFG

        status_df = self.append_profit_loss(status_df)
        filter_status_df = self.get_filter_status_df(status_df, CFG)
        selling_orders = self.get_order_from_df(filter_status_df)

        return selling_orders


def merge_order(buying_orders: list, selling_orders: list) -> list[tuple[str, int]]:
    total_order = buying_orders + selling_orders
    total_order_df = pd.DataFrame(total_order, columns=["SYMBOL", "ORDER"])
    symbols_and_orders = list(
        total_order_df.groupby("SYMBOL").sum().squeeze().astype(int).to_dict().items()
    )
    return symbols_and_orders


In [ ]:
# TRADE_FUNC
import random
import logging
import datetime as dt

import pandas as pd
import kquant as kq
from sklearn.preprocessing import MinMaxScaler

def trade_func(
    date: dt.date,
    dict_df_result: dict[str, pd.DataFrame],
    dict_df_position: dict[str, pd.DataFrame],
    logger: logging.Logger,
) -> list[tuple[str, int]]:
    """
    CFG : trade_fun의 파라미터 조정
    """
    CFG = {
        "cash_percentage": 0.75,  # 1일 투자 금액 (보유 현금 * 0.75)
        "buying_order_n": 8,  # 1일 구매 stock 종류수
    }
    """
    STATUS_LOADER
    """
    status_loader = STATUS_LOADER(dict_df_result, dict_df_position)

    current_cash = status_loader.get_current_cash()
    invest_money = current_cash * CFG["cash_percentage"]

    status_df = status_loader.get_status_df()
    """
    SYMBOL_LOADER
    """
    symbol_loader = SYMBOL_LOADER()
    total_symbols = symbol_loader()

    """
    SYMBOL_SECTOR_PROCESSOR
    """
    symbol_sector_processor = SYMBOL_SECTOR_PROCESSOR(total_symbols)
    sampled_symbol_df = symbol_sector_processor()

    """
    SCORE_PROCESSOR
    """
    sectors = sorted(set(sampled_symbol_df["SECTOR"]))

    score_df_list = list()
    for sector in sectors:
        _sector_symbol_df = sampled_symbol_df[sampled_symbol_df["SECTOR"] == sector]
        _symbols = sorted(set(_sector_symbol_df["SYMBOL"]))

        score_processor = SCORE_PROCESSOR(_symbols, date)
        _score_df = score_processor()
        score_df_list.append(_score_df)

    score_df = pd.concat(score_df_list)
    """
    BUYING_ORDER_PROCESSOR
    """
    buying_order_processor = BUYING_ORDER_PROCESSOR(
        score_df, invest_money, status_df, CFG["buying_order_n"]
    )
    buying_orders = buying_order_processor()

    selling_order_processor = SELLING_ORDER_PROCESSOR(status_df)
    selling_orders = selling_order_processor()

    symbols_and_orders = merge_order(buying_orders, selling_orders)
    return symbols_and_orders


In [ ]:
import datetime as dt
dict_df_result_1, dict_df_position_1, logger = kq.backtest_stock_port_daily(
    trade_func,
    "2023-08-22",
    "2023-08-22",
    init_cash=1_000_000_000,
    return_position=True,
    return_logger=True,
)

In [ ]:
date = dt.date(2023, 8, 23)
symbols_and_orders = trade_func(
    date,
    dict_df_result_1,
    dict_df_position_1,
    logger,
)
dict_df_result_2, dict_df_position_2 = kq.backtest_update_stock_port_daily(
    symbols_and_orders,
    date,
    dict_df_result_1,
    dict_df_position_1,
)

In [ ]:
date = dt.date(2023, 8, 24)

symbols_and_orders = trade_func(
    date,
    dict_df_result_2,
    dict_df_position_2,
    logger,
)

dict_df_result_3, dict_df_position_3 = kq.backtest_update_stock_port_daily(
    symbols_and_orders,
    date,
    dict_df_result_2,
    dict_df_position_2,
)

In [ ]:
date = dt.date(2023, 9, 22)

dict_df_result_4, dict_df_position_4 = kq.backtest_update_stock_port_daily(
    [],
    date,
    dict_df_result_3,
    dict_df_position_3,
)
result = dict_df_result_4["TOTAL"]["TOTAL_VALUE"].tail(1).values[0]
result

In [ ]:
result

In [ ]:
import kquant as kq

# loop
dict_df_result = kq.backtest_stock_port_daily(
    trade_func,
    "2023-08-22",  # 실제 심사에서는 투자기간 시작일
    "2023-09-10",  # 실제 심사에서는 투자기간 종료일
    init_cash=1_000_000_000,  # 10억원
)